In [1]:
%pip install -r ../requirements.txt

import sqlite3
import glob
import os
from lxml import etree
from import_sdc_form_sqlite import process_xml
from import_vol_v_message_sqlite import import_data_from_hl7

DB_FILEPATH = "public/sdc_cdm.db"

Note: you may need to restart the kernel to use updated packages.


In [2]:
if os.path.exists(DB_FILEPATH):
    print(f"Deleting existing database file: {DB_FILEPATH}")
    os.remove(DB_FILEPATH)

# Connect to the SQLite database (creates the database file if it doesn't exist)
conn = sqlite3.connect(DB_FILEPATH)
cursor = conn.cursor()
print("Connected to the SQLite database.")

# Get a list of all .sql files in the ./ddl directory
sql_files = glob.glob('../database/ddl/sqlite/*.sql')

# Sort the files alphanumerically
sql_files.sort()

# Display the files to be executed
print("SQL files to be executed in order:")
for file in sql_files:
    print(os.path.basename(file))

# Execute each SQL file
for sql_file in sql_files:
    print(f"\nExecuting {os.path.basename(sql_file)}...")
    with open(sql_file, 'r') as file:
        sql_script = file.read()
    try:
        cursor.executescript(sql_script)
        print(f"Finished executing {os.path.basename(sql_file)}.")
    except Exception as e:
        print(f"An error occurred while executing {os.path.basename(sql_file)}: {e}")

# Commit changes and close the connection
conn.commit()
conn.close()

print("Database changes committed and connection closed.")

Deleting existing database file: public/sdc_cdm.db
Connected to the SQLite database.
SQL files to be executed in order:
1_OMOPCDM_sqlite_5.4_ddl.sql
2_OMOPCDM_sqlite_5.4_primary_keys.sql
3_OMOPCDM_sqlite_5.4_indices.sql
4_OMOPCDM_sqlite_5.4_constraints.sql
5_SDC_sqlite.sql

Executing 1_OMOPCDM_sqlite_5.4_ddl.sql...
Finished executing 1_OMOPCDM_sqlite_5.4_ddl.sql.

Executing 2_OMOPCDM_sqlite_5.4_primary_keys.sql...
Finished executing 2_OMOPCDM_sqlite_5.4_primary_keys.sql.

Executing 3_OMOPCDM_sqlite_5.4_indices.sql...
Finished executing 3_OMOPCDM_sqlite_5.4_indices.sql.

Executing 4_OMOPCDM_sqlite_5.4_constraints.sql...
Finished executing 4_OMOPCDM_sqlite_5.4_constraints.sql.

Executing 5_SDC_sqlite.sql...
Finished executing 5_SDC_sqlite.sql.
Database changes committed and connection closed.


In [3]:
conn = sqlite3.connect(DB_FILEPATH)
cursor = conn.cursor()

namespaces = {
    "sdc": "urn:ihe:qrph:sdc:2016",
    "xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "xsd": "http://www.w3.org/2001/XMLSchema",
}

# Import SDC templates
v2_messages = glob.glob(os.path.join("../sample_data/sdc_templates", "*.xml"))

# Iterate over each XML file
for message in v2_messages:
    # Read the XML content
    with open(message, "r", encoding="utf-8") as file:
        message_str = file.read()

    tree = etree.parse(message)
    root = tree.getroot()

    form_design = root.find("sdc:FormDesign", namespaces)
    if form_design is None:
        form_design = root

    if form_design is None:
        print(f"No Form Design found in {os.path.basename(message)}")
        continue
    print(f"Form Design: {form_design}")

    # Extract column values from the XML content
    sdcformdesignid = form_design.get("ID")
    baseuri = form_design.get("baseURI")
    lineage = form_design.get("lineage")
    version = form_design.get("version")
    fulluri = form_design.get("fullURI")
    formtitle = form_design.get("formTitle")
    sdc_xml = etree.tostring(form_design).decode("utf-8")
    doctype = "FD"  # TODO: Parse from fullURI

    # Insert the data into the database
    try:
        cursor.execute(
            """
            INSERT INTO main.templatesdcclass (
                sdcformdesignid, baseuri, lineage, version, fulluri, formtitle, sdc_xml, doctype
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """,
            (
                sdcformdesignid,
                baseuri,
                lineage,
                version,
                fulluri,
                formtitle,
                sdc_xml,
                doctype,
            ),
        )
        print(f"Inserted data from {os.path.basename(message)} into the database.")
    except Exception as e:
        print(
            f"An error occurred while inserting data from {os.path.basename(message)}: {e}"
        )

# Commit the changes
conn.commit()
print("All XML data has been inserted into the database.")

cursor.execute("SELECT COUNT(*) FROM main.templatesdcclass;")
count = cursor.fetchone()[0]
print(f"\nTotal records inserted: {count}")

# Commit changes and close the connection
conn.close()

Form Design: <Element {urn:ihe:qrph:sdc:2016}FormDesign at 0x727f2bc3c800>
Inserted data from Kidney.Bx.149_2.005.001.REL_sdcFDF.xml into the database.
Form Design: <Element {urn:ihe:qrph:sdc:2016}FormDesign at 0x727f2b25f200>
Inserted data from Neuroblastoma.Res.153_4.000.000.REL_sdcFDF.xml into the database.
Form Design: <Element {urn:ihe:qrph:sdc:2016}FormDesign at 0x727f2bc15a80>
Inserted data from ExGonad.GCT.Bx.369_2.000.000.REL_sdcFDF.xml into the database.
Form Design: <Element {urn:ihe:qrph:sdc:2016}FormDesign at 0x727f2b30e300>
Inserted data from Stomach.NET.Res.201_3.000.001.REL_sdcFDF.xml into the database.
Form Design: <Element {urn:ihe:qrph:sdc:2016}FormDesign at 0x727f4c91c6c0>
Inserted data from Skin.Merkel.Res.198_2.003.001.REL_sdcFDF.xml into the database.
Form Design: <Element {urn:ihe:qrph:sdc:2016}FormDesign at 0x727f2b320800>
Inserted data from Prostate.Needle.Specimen.Bx.508_1.001.001.REL_sdcFDF.xml into the database.
Form Design: <Element {urn:ihe:qrph:sdc:2016}

In [4]:
# Import SDC forms
conn = sqlite3.connect(DB_FILEPATH)
cursor = conn.cursor()

# Import SDC templates
v2_messages = glob.glob(os.path.join("../sample_data/sdc_xml", "*.xml"))

# Iterate over each XML file
for message in v2_messages:
    # Read the XML content
    with open(message, "r", encoding="utf-8") as file:
        message_str = file.read()

    tree = etree.parse(message)
    root = tree.getroot()
    process_xml(root, cursor)

cursor.close()
conn.commit()
conn.close()


INFO:crud_sqlite:Inserting new TemplateSdcClass: {'sdcformdesignid': 'Adrenal.Bx.Res.129_3.002.011.RC1_sdcFDF', 'baseuri': 'cap.org', 'lineage': 'Adrenal.Bx.Res.129', 'version': '3.002.011.RC1', 'fulluri': '_baseURI=cap.org&_lineage=Adrenal.Bx.Res.129&_version=3.002.011.RC1&_docType=sdcFDF', 'formtitle': 'ADRENAL GLAND', 'sdc_xml': '<FormDesign xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns="urn:ihe:qrph:sdc:2016" order="0" ID="Adrenal.Bx.Res.129_3.002.011.RC1_sdcFDF" baseURI="cap.org" fullURI="_baseURI=cap.org&amp;_lineage=Adrenal.Bx.Res.129&amp;_version=3.002.011.RC1&amp;_docType=sdcFDF" filename="Adrenal.Bx.Res.129_3.002.011.RC1_sdcFDF.xml" lineage="Adrenal.Bx.Res.129" formTitle="ADRENAL GLAND" version="3.002.011.RC1">\n    <Property name="Copyright" type="CAPeCC_static_text" styleClass="copyright" order="1" propName="Copyright" val="(c) 2018 College of American Pathologists.  All rights reserved.  License required for use."/

Form Design: <Element {urn:ihe:qrph:sdc:2016}FormDesign at 0x727f2b320fc0>


In [5]:
# Import NAACCR data
conn = sqlite3.connect(DB_FILEPATH)
cursor = conn.cursor()

v2_messages = glob.glob(os.path.join("../sample_data/naaccr_v2", "*.hl7"))

# Iterate over each XML file
for message in v2_messages:
    try:
        # Read the XML content
        with open(message, "r", encoding="utf-8") as file:
            message_str = file.read()

        import_data_from_hl7(cursor, message_str)
    except Exception as e:
        print(f"Error processing message: {message}: {e}")
        continue

cursor.close()
conn.commit()
conn.close()

INFO:crud_sqlite:Inserting new TemplateSdcClass: {'sdcformdesignid': None, 'baseuri': None, 'lineage': None, 'version': '3.007.011.1000043', 'fulluri': None, 'formtitle': 'ADRENAL GLAND', 'sdc_xml': None, 'doctype': None}
INFO:crud_sqlite:Successfully added: {'sdcformdesignid': None, 'baseuri': None, 'lineage': None, 'version': '3.007.011.1000043', 'fulluri': None, 'formtitle': 'ADRENAL GLAND', 'sdc_xml': None, 'doctype': None, 'pk': 103}
INFO:crud_sqlite:Successfully added: {'templateinstanceversionguid': None, 'templateinstanceversionuri': None, 'templatesdcfk': 103, 'instanceversiondate': None, 'diagreportprops': None, 'surgpathid': None, 'personfk': None, 'encounterfk': None, 'practitionerfk': None, 'reporttext': None, 'pk': 2}
INFO:crud_sqlite:Successfully added: {'templateinstanceclassfk': 2, 'parentinstanceguid': None, 'section_id': None, 'section_guid': None, 'q_text': 'Tumor Site', 'q_instanceguid': None, 'q_id': '2118.1000043', 'li_text': '2119.1000043', 'li_id': 'Adrenal gla

Message type: ORU^R01^ORU_R01
Message profile: VOL_V_40_ORU_R01^NAACCR_CP
Report type: 60568-3^SYNOPTIC REPORT^LN
First OBX identifier: 60573-3^Report template source^LN
Document source style: CAP eCC
Template ID: 129.1000043^ADRENAL GLAND^CAPECC
! Form Title: ADRENAL GLAND
! Version ID: 3.007.011.1000043
Q ID: 2118.1000043
Q Text: Tumor Site
Q ID: 820603.1000043
Q Text: Procedure
Q ID: 52756.1000043
Q Text: Specimen Laterality
Q ID: 59852.1000043
Q Text: Histologic Subtype (Notes C, D)
Q ID: 820395.1000043
Q Text: Mitotic Tumor Grade (required for adult patients only) (Notes C, D)
Q ID: 2129.1000043
Q Text: Tumor Size (Notes E, F)
@@@ Observation sub ID: 2131
Q ID: 2129.1000043
Q Text: Tumor Size (Notes E, F)
Observation units: cm^CentiMeter^UCUM
@@@ Observation sub ID: +2131.1000043
Q ID: 820398.1000043
Q Text: Adrenal Gland Weight (Note G)
@@@ Observation sub ID: 44761
Q ID: 820398.1000043
Q Text: Adrenal Gland Weight (Note G)
@@@ Observation sub ID: 44761
@@@@@ Observation sub ID a

In [6]:
import threading
import http.server
import socketserver
from contextlib import contextmanager

PORT = 8000
DIRECTORY = './public'

class Handler(http.server.SimpleHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=DIRECTORY, **kwargs)

@contextmanager
def run_server():
    httpd = socketserver.TCPServer(('', PORT), Handler)
    httpd.allow_reuse_port = True
    server_thread = threading.Thread(target=httpd.serve_forever)
    server_thread.daemon = True
    server_thread.start()
    print(f"Serving at port {PORT}")
    try:
        yield
    finally:
        httpd.shutdown()
        httpd.server_close()
        httpd.socket.close()
        print("Server stopped")

with run_server():
    # Run until interrupt
    while True:
        pass

Serving at port 8000


127.0.0.1 - - [13/Nov/2024 19:23:42] "GET / HTTP/1.1" 304 -
127.0.0.1 - - [13/Nov/2024 19:23:42] "GET /sql-wasm-debug.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Nov/2024 19:23:42] "GET /index.css HTTP/1.1" 304 -
127.0.0.1 - - [13/Nov/2024 19:23:42] "GET /sql-wasm-debug.wasm HTTP/1.1" 304 -
127.0.0.1 - - [13/Nov/2024 19:23:42] code 404, message File not found
127.0.0.1 - - [13/Nov/2024 19:23:42] "GET /favicon.ico HTTP/1.1" 404 -
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 43920)
Traceback (most recent call last):
  File "/usr/lib/python3.11/http/server.py", line 726, in send_head
    f = open(path, 'rb')
        ^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: './public/favicon.ico'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, clien

Server stopped


KeyboardInterrupt: 